In [29]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import requests
import random

load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')

CHAT_MODEL = 'gpt-4o-mini'
print('ready', openai_key[:10] + "**", CHAT_MODEL)

ready sk-proj-XV** gpt-4o-mini


In [30]:
class PostGenerator:


    def __init__(self, chat_model=CHAT_MODEL):
        openai_key = os.environ['OPENAI_API_KEY']
        client = OpenAI(
            api_key=openai_key,  # this is also the default, it can be omitted
        )
        self.client = client
        self.chat_model = chat_model
        self.max_tokens = 160

    # https://platform.openai.com/docs/guides/text-generation#text-generation-models
    def create_response_for_messages(self, messages):
        response = self.client.chat.completions.create(
            model=self.chat_model,
            messages=messages,
            max_tokens=self.max_tokens
        )
        return response.choices[0].message.content


    def generate_tweet(self, user_background, target_audience, event):
        prompt = f"""Generate a formatted tweet (with newlines) for someone with a background in {user_background}, targeting {target_audience}. Current event: {event}.
        -- Using this event for the post take a hard stance or side on the topic with a short explanation as to why. No emojis or hashtags.
        Compare it to something in the past"""
        messages = [
            {'role': 'developer', 'content': 'You are a social media manager responsible for making posts. Do not make these posts sound robotics. Insert abbreviations and language to make the post sound human.'},
            {'role': 'user', 'content': prompt}
        ]
        tweet = self.create_response_for_messages(messages)
        return tweet

    def rate_tweet(self, tweet):
        prompt = f"Rate the following tweet on its effectiveness from 1-10 and provide the top 3 suggestions for improvement:\n\n{tweet}"
        messages = [
            {'role': 'user', 'content': prompt}
        ]
        response = self.create_response_for_messages(messages)
        return response

    def improve_tweet(self, tweet, suggestions):
        prompt = f"Improve the following tweet based on these suggestions:\n\nTweet: {tweet}\n\nSuggestions: {suggestions}. The post should not have emojis or hashtags."
        messages = [
            {'role': 'user', 'content': prompt}
        ]
        response = self.create_response_for_messages(messages)
        return response


In [31]:
def format_article(a):
    text = f"{a['title']} {a["description"]}"
    return text.replace('[Removed]', '')

def get_current_events(theme):
    response = requests.get('https://newsapi.org/v2/everything', params={
        'apiKey': os.getenv('NEWS_API_KEY'),
        'q': theme,
        'pageSize': 5
    })
    articles = response.json().get('articles', [])
    articles = [format_article(a) for a in articles]
    return articles

current_events = None

In [ ]:
# Example usage
background = "software, engineering, science, entrepreneurship"
interests = "coding, open-source projects, hackathons, and startups"
target_audience = "developers and entrepreneurs"
theme = "technology, science"

if not current_events:
  current_events = get_current_events(theme)
  print('fetched events', current_events)
else:
  print('using cached events', current_events)

fetched events ["Can RFK Jr make America's diet healthy again? Kennedy's plans to ban dyes in cereals and fluoride in water could get major pushback from the food industry.", 'Biden backs calls to ban congressional stock trading President Joe Biden lent his voice to the growing bipartisan calls on the Hill to end stock trading for congressional members.', 'The Download: words of wisdom from the departing White House tech advisor, and controversial AI manga translation This is today’s edition of\xa0The Download,\xa0our weekday newsletter that provides a daily dose of what’s going on in the world of technology. What the departing White House chief tech advisor has to say on AI President Biden’s administration will end within two m…', 'China sends cloud powered by homebrew Loongson CPUs into space Plus: Korea cracks down on Temu; US, Vietnam, sign infosec pact; India extends controversial hardware import law; and more\nAsia In Brief Chinese chip designer Loongson last Friday announced its

In [33]:
random_index = 0 if len(current_events) == 1 else random.randint(0, len(current_events) - 1)

event = current_events[random_index]
print('selected event', event)

generator = PostGenerator()
tweet = generator.generate_tweet(background, target_audience, event)
print('generated tweet:\n', tweet)

selected event House Passes Bill That Bans Transgender Care for Military Members’ Children The House approved a $895 billion bill, which included a provision prohibiting gender-affirming care for transgender children of service members
generated tweet:
 The recent House vote to ban gender-affirming care for military members’ children is a step backward for equality and human rights. 

We’ve seen similar restrictions before, like the "Don't Ask, Don't Tell" policy, which marginalized LGBTQ+ service members. 

Every child, regardless of their parents' service, deserves access to care that supports their identity. Our future leaders should not be silenced or sidelined. Let's advocate for inclusivity!
